# CLASIFICACIÓN DE RIPPLES


-Algoritmo basado en aprendizaje supervisado (SVM) para la detección  de "ripples", LFPs o picos de actividad cerebral en las
series temporales registradas por 8 delectrodos localizados en el hipotálamo de un ser humano. Para más información
https://thebraincodegames.github.io/ .

-Cada dispositivo registra un valor de potencial eléctrico cada (1/30.000) s. Al valor del denominador se le conoce como
"frecuencia de sampleo".

-Para "alimentar" los datos recabados a nuestro algoritmo se han dividido los muestreos en segmentos de "A" instantes.
Cabe recordar que un instante cada 1/30.000 segundos, luego los segmentos tendrán una duración de A/30.000 segundos.
El algoritmo evalua cada segmento en base a sus A instantes, para decidir si contiene o no un ripple.
A este conjunto de segmentos que constituyen el "input" de nuestro algoritmo, se ha referenciado como 'X'.

-Para determinar el valor óptimo del ancho del segmento, el hiperparámetro "A", se han compararon los resultados 
del algoritmo para distintos valores situados entre el periodo de oscilación de un "ripple" y la duración media de uno.

-Dado que se trata de aprendizaje supervisado, se necesita un conjunto de etiquetas elaboradas por un experto que informen
al algoritmo durante su aprendizaje sobre lo que es y no es un ripple. A este conjunto de etiquetas, se ha referenciado como 'y'.

# IMPORTACIONES


In [3]:
# Bibliotecas instaladas
'''
pip install imblearn
pip install numpy
pip install matplotlib
pip install sklearn
'''

'\npip install imblearn\npip install numpy\npip install matplotlib\npip install sklearn\n'

In [4]:
import bz_LoadBinary
import bcg_auxiliary

import numpy as np
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler

# FUNCIONES AUXILIARES

In [5]:
''' 
Funcionalidad: Cargo la sesion de entrenamiento.
Parámetros: Ruta relativa o abosulta de la sesion de entrenamiento. Imprime metadatos de interés de lo cargado.
Return: 
  X --> NumPy-Array de dimesiones (N-instantes, 8-canales) que contiene el valor de cada canal
  para los 8 instantes.
  y --> NumPy-Array de dimensiones (M-ripples, 2) que contiene el valor tiempo de inicio y tiempo
  de fin de cada ripple de la sesión.
  fs --> Frecuencia de sampleo del archivo
'''

def load_data(path_archivo):
  X, fs, session_name = bcg_auxiliary.load_data(path_archivo)
  y = bcg_auxiliary.load_ripples_tags(path_archivo, fs=30000)

  print("Numero de canales ", len(X[0])) # Numero de canales
  print("Numero de samples ", len(X)) # Numero de canales
  print("Numero de ripples ", len(y), "\n") # Numero de ripples

  return X, y, fs


In [6]:
'''
Funcionalidad: Agrupar los elemetos de un NumPyArray en conjunto de tamaño A.
Parámetros:
  X --> NumPy-Array de dimesiones (N-instantes, 8-canales) que contiene el valor de cada canal
  para los 8 instantes.
  A --> Tamaño de las agrupaciones.
Return: NumPy
'''
def agrupar_x(x, A):
  #elimino los sobrantes datos
  if(-int(np.shape(x)[0]%A)!= 0):
    x=x[:-int(np.shape(x)[0]%A)]
  #calculo en cuantos segmentos tendré que dividir el data set
  x_dividido=np.array_split(x, int(abs(np.shape(x)[0]/A)))
  return x_dividido


In [7]:
''' Funcionalidad: A partir de una lista binaria de longitud N (signal) se genera una lista binaria
de longitud N/A que indica si existe al menos un 1 cada A elementos de la lista original. En el contexto de la hackaton,
dado que contamos con una lista que informa si existe ripple o no a cada instante (len(instante)=1/frecuencai de sampleo)
esta función permite definir en cuales de los A segmentos en los que se ha divido la serie temporal existe un ripple.
El criterio elegido ha sido que si al menos un instante de unsigmento forma parte de un ripple, el segmento contiene uno.
Parámetros:
  -signal: Lista binaria de longitud N.
  -A: Tamaño del segmento.
Return: Lista binaria de longitud len(signal)/A que indica si existe "ripple" en cada uno de los segmentos en los
que se hadividido la serie.
 '''
def binary_to_binary_grouped(signal, A):
  lista = []
  for i in range(int(len(signal)/A)):
    if(np.any(signal[i*A:i*A+A] == 1) == True):
      lista = lista + [1]
    else:
      lista = lista + [0]
  return lista



In [8]:
'''
Funcionalidad: Inversa a la bcg_auxiliary.get_ripples_tags_as_signal(X, ripples, fs). Transforma una señal/array binario
en una lista con el valor tiempo de inicio y tiempo de fin de cada ripple de la sesión.
Parámetros:
  signal --> Array binario que indica si existe o no "ripple" en cada segmento en los que se ha fragmentado la sesión.
  A --> Ancho del segmento en instantes (1/fs).
  fs --> Frecuencia de sampleo de la sesión.
'''
def signal_to_time(signal, A, fs):
  # Toma signal, como devuelve la función del hackathon
  pred_ripples = [[],[]]

  if signal[0] == 1: # Por si empieza un ripple en cero
    pred_ripples[0].append(0) 

  for idx in range(len(signal)-1):

    if signal[idx] == 0 and signal[idx+1] == 1: # Esto significa que he encontrado el comienzo de un ripple
      pred_ripples[0].append((idx+1)*A/fs)

    if signal[idx] == 1 and signal[idx+1] == 0: # Esto significa que he encontrado el final de un ripple
      pred_ripples[1].append((idx+1)*A/fs)
  
  if len(pred_ripples[1]) != len(pred_ripples[0]): # Por si pillo un ripple en el final pongo un ultimo tiempo
    pred_ripples[1].append(len(signal)*A/fs)

  return np.array(pred_ripples).transpose() # Transpose para que las columnas sean filas, como nos lo dan



# PROCESADO

In [9]:
'''
Funcionalidad: Adaptar la información disponible a las características del algorimto.
Parámetros:
  X --> NumPy-Array de dimesiones (N-instantes, 8-canales) que contiene el valor de cada canal
  para los 8 instantes.
  y --> NumPy-Array de dimensiones (M-ripples, 2) que contiene el valor tiempo de inicio y tiempo
  de fin de cada ripple de la sesión
  A --> Ancho de los segmentos en los que se agrupan los valores registrados (X).
  fs --> Frecuencia de Sampleo
Return:
  X_1 --> NumPy-Array (matriz) de dimensiones ( N/A [segmentos], A*(8_canales) [instantes por segmento] ).
  y_1 --> NumPy-Array de dimensiones (vector) ( N/A [segmentos],) binario que indica si el segmento i-ésimo es ripple (1) o no (0).
'''
def procesado_pre_entrenamiento(X_1, y_1,fs,A):
  #Procesamiento de la y
  y_1 = bcg_auxiliary.get_ripples_tags_as_signal(X_1, y_1,fs) # (m-ripples,2)=[[t0,t1]...[tn-1,tn]] --> n valores binarios que indican si en ese instante (1/30.000 segundos) hay (1) o no (0) ripple.
  y_1 = binary_to_binary_grouped(signal=y_1, A=A) #Agrupación en segmentos:

  #Procesamiento de la X

  X_1 = X_1/np.max(X_1) #Normalizacion
  X_1 = agrupar_x(X_1, A) #Agrupa los instantes en matrices de Ax8. Un tensor no válido para el algoritmo utilizado.
  X_1 = np.reshape(X_1, [len(X_1), 8*len(X_1[0])]) #Formato "dimensional" aceptado por el algoritmo a utilizar

  return X_1, y_1, fs

In [10]:
# Selección de hiperparámetros
A = int(1500) #Ancho de ventana --> 500 puntos por segmento analizado
fs = 30000 #Frecuencia de sampleo original

In [11]:
#Carga de información
train_data_path_1="data/train_1"
train_data_path_2="data/train_2"

X_1_o, y_1_o, fs = load_data(train_data_path_1)
X_2_o, y_2_o, fs = load_data(train_data_path_2)

#Ambos tienen la misma frecuencia de sampleo: 30.000
X_1, y_1, _ = procesado_pre_entrenamiento(X_1_o, y_1_o, fs, A)
X_2, y_2, _ = procesado_pre_entrenamiento(X_2_o, y_2_o, fs, A)

#Concatenado de ambos sets
X = np.concatenate((X_1, X_2))
y = np.concatenate((y_1, y_2))

#Undersampleo para equilibrar la muestra (hay más de un 90% de negativos)
rus = RandomUnderSampler()
X, y, = rus.fit_resample(X, y)

Numero de canales  8
Numero de samples  31087616
Numero de ripples  485 

Numero de canales  8
Numero de samples  71965696
Numero de ripples  1309 



MemoryError: Unable to allocate 632. MiB for an array with shape (6898, 12000) and data type float64

In [ ]:
np.save(f"data/X_train_{A}.npy", X) # Guardo los datos de entrenamiento
np.save(f"data/y_train_{A}.npy", y) 

# ENTRENAMIENTO

In [12]:
# Si se ha ejecutado la parte de preprocesamiento no es necesario volver a ejecutarla

X = np.load(f"data/X_train_{A}.npy")
y = np.load(f"data/y_train_{A}.npy")


MemoryError: Unable to allocate 632. MiB for an array with shape (82776000,) and data type float64

In [ ]:
from sklearn import svm
import pickle

clf = svm.SVC()
clf.fit(X, y)

pred_ripples = clf.predict(X)

In [ ]:
pkl_filename = f"modelos/model_SVM_{A}.pkl"

#Guardar resultados de entrenamiento
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)
    
'''
# Cargar Datos de entrenamiento
with open(pkl_filename, 'rb') as file:
    clf = pickle.load(file)
'''

"\n# Cargar Datos de entrenamiento\nwith open(pkl_filename, 'rb') as file:\n    clf = pickle.load(file)\n"

In [ ]:
matches = [i == j for i,j in zip(pred_ripples, y)]
acc = matches.count(True)/len(matches)

print("Precisión en el training set: ", acc)

Precisión en el training set:  0.8756161206146709


# VALIDACIÓN

In [ ]:
val_data_path="data/val_2"
X, y, _ = load_data(val_data_path)

Numero de canales  8
Numero de samples  22326272
Numero de ripples  1064 



In [ ]:
#Pre-procesado de la X
X = X/np.max(X) #Normalizo
X = agrupar_x(X, A) #Agrupa los instantes en A/N matrices de Ax8. Un tensor no válido para el algoritmo utilizado.
X = np.reshape(X, [len(X), 8*len(X[0])]) #Formato "dimensional" aceptado por el algoritmo a utilizar: ( M x L ) = ( A/N segmentos x A*(8_canales) ).

true_ripples = y

#Predicción
pred_ripples = clf.predict(X)
pred_ripples = signal_to_time(pred_ripples,A,fs)


In [ ]:
print("Verdaderos ripples: \n", true_ripples)
print("Predicción ripples: \n", pred_ripples) 

Verdaderos ripples: 
 [[1.60466667e-01 2.01500000e-01]
 [3.01100000e-01 3.33333333e-01]
 [4.31966667e-01 4.93466667e-01]
 ...
 [7.40693600e+02 7.40753267e+02]
 [7.40821600e+02 7.40918100e+02]
 [7.42618767e+02 7.42658900e+02]]
Predicción ripples: 
 [[1.0000e-01 2.0000e-01]
 [4.5000e-01 5.5000e-01]
 [6.5000e-01 7.0000e-01]
 ...
 [7.4260e+02 7.4270e+02]
 [7.4295e+02 7.4300e+02]
 [7.4415e+02 7.4420e+02]]


In [ ]:
print("P, R, F1: ", bcg_auxiliary.get_score(true_ripples, pred_ripples, threshold=0.1))

P, R, F1:  (0.4793779580797836, 0.6663533834586466, 0.5576091230829728)


# Test

In [ ]:
def test(test_data_path):
    X, y, _ = load_data(val_data_path)

    #Pre-procesado de la X
    X = X/np.max(X) #Normalizo
    X = agrupar_x(X, A) #Agrupa los instantes en A/N matrices de Ax8. Un tensor no válido para el algoritmo utilizado.
    X = np.reshape(X, [len(X), 8*len(X[0])]) #Formato "dimensional" aceptado por el algoritmo a utilizar: ( M x L ) = ( A/N segmentos x A*(8_canales) ).

    true_ripples = y

    #Predicción
    pred_ripples = clf.predict(X)
    pred_ripples = signal_to_time(pred_ripples,A,fs)
    _, _, session_name = bcg_auxiliary.load_data(val_data_path)

    bcg_auxiliary.write_results("resultados/", session_name, 13, pred_ripples)

In [ ]:
test("data/test1")
test("data/test2")

NameError: name 'load_data' is not defined